In [ ]:
import os
from google.colab import output
from google.colab import files

file_selector = files.upload()

if not file_selector:
  print("No file selected.")

uploaded_file = next(iter(file_selector.values()))

try:
  if not uploaded_file.name.endswith(".xlsx"):
    print("Please select an .xlsx file.")
except:
  print("Please check file type")

with open("/content/data.xlsx", "wb") as f:
  f.write(uploaded_file)

# Print a success message
print("File uploaded and saved as data.xlsx.")

Saving CIE L1_Aug2023 semester_OAT data.xlsx to CIE L1_Aug2023 semester_OAT data (1).xlsx
Please check file type
File uploaded and saved as data.xlsx.


In [1]:
!mkdir "/content/Model/"
!clear
!wget https://github.com/DoggoWoofWoof/CIE-Candidate-Selection/raw/main/Models/model-bert-use-this.hdf5
!clear
!mv "/content/model-bert-use-this.hdf5" "/content/Model/model-bert-use-this.hdf5"
!clear
!wget https://github.com/DoggoWoofWoof/CIE-Candidate-Selection/raw/main/Models/model_weights_and_optimizer.pickle
!clear
!mv "/content/model_weights_and_optimizer.pickle" "/content/Model/model_weights_and_optimizer.pickle"
!clear
!wget https://github.com/DoggoWoofWoof/CIE-Candidate-Selection/raw/main/Models/random_forest_model.joblib
!clear
!mv "/content/random_forest_model.joblib" "/content/Model/random_forest_model.joblib"
!clear
!wget https://github.com/DoggoWoofWoof/CIE-Candidate-Selection/raw/main/Models/vectorizer.joblib
!clear
!mv "/content/vectorizer.joblib" "/content/Model/vectorizer.joblib"
!clear

--2024-04-20 16:08:19--  https://github.com/DoggoWoofWoof/CIE-Candidate-Selection/raw/main/Models/model-bert-use-this.hdf5
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DoggoWoofWoof/CIE-Candidate-Selection/main/Models/model-bert-use-this.hdf5 [following]
--2024-04-20 16:08:19--  https://raw.githubusercontent.com/DoggoWoofWoof/CIE-Candidate-Selection/main/Models/model-bert-use-this.hdf5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 418600 (409K) [application/octet-stream]
Saving to: ‘model-bert-use-this.hdf5’

model-bert-use-this 100%[===================>] 408.79K  --.-KB/s    in 0.02s   

2

Verify that the excel file has been added to the current directory.

> Need to verify later if the pickle files of both @Sairam & @Swastik are added to the current directory

In [ ]:
!echo "Please Be Patient"
!pip install tensorflow_text -q
!pip install --upgrade tf-keras -q
!pip install aspose-cells -q
!echo "Thank You For Your Patience"

Please Be Patient
Thank You For Your Patience


In [ ]:
#we are selecting Columns based of index keep in mind that index of Level1 can be repeated in Level2 but do not repeat index for deleted columns anywhere it is error
import pandas as pd
data_df = pd.read_excel('/content/data.xlsx').fillna('')
print("Column names with their index:\n")
for idx, col in enumerate(data_df.columns):
    print(f"{idx:<3}: {col[:20].ljust(20, ' ')}...")
cols_to_del = [int(x.strip()) for x in input("Enter the column numbers to be deleted (comma-separated): ").split(',')]
level1_idx, level2_idx = [int(i) for i in input(f"Enter the indexes for Level1 (comma-separated): ").split(',')], [int(i) for i in input(f"Enter the indexes for Level2 (comma-separated): ").split(',')]

Column names with their index:

0  : Status              ...
1  : Branch              ...
2  : Semester            ...
3  : Do you understand an...
4  : Are you willing to c...
5  : The assessment for t...
6  : We strongly recommen...
7  : Give one/top reason ...
8  : What (one/top thing)...
9  : How would you descri...
10 : Excluding studies/ac...
11 : Briefly describe the...
12 : What does entreprene...
13 : Cite your 3 key stre...
14 : Cite your 3 key weak...
15 : According to you, wh...
16 : According to you, wh...
17 : What are the top 3 q...
18 : What 3 qualities/att...
19 : How would you rate y...
20 : Give an example of a...
21 : How well do you deal...
22 : How easy or difficul...
23 : On a scale of 1 (Low...
24 : Can you give one exa...
25 : Name any Indian entr...
26 : Bonus question (extr...
27 : Optional question: A...
Enter the column numbers to be deleted (comma-separated): 0,1,2,3,4,5,6
Enter the indexes for Level1 (comma-separated): 12,24,7,8
Enter the indexes for Leve

In [ ]:
#AI detection works for only Level 1 questions
questions = [data_df.columns[i] for i in level1_idx]
full_dataframe=data_df.copy()

AI Component Detection Below

In [ ]:
#run this cell once before predicting values (loading the model)
#Accuracy of the model = 82-86%
import tensorflow
import pickle
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_text as text  # Registers the ops.

import numpy as np


class BertEncoder(tensorflow.keras.layers.Layer):
    def __init__(self, preprocess_path, model_path, **kwargs):
        super(BertEncoder, self).__init__(**kwargs)
        self.preprocessor = hub.KerasLayer(preprocess_path)
        self.encoder = hub.KerasLayer(model_path, trainable=True)

    def call(self, inputs):
        encoder_inputs = self.preprocessor(inputs)
        outputs = self.encoder(encoder_inputs)
        pooled_output = outputs['pooled_output']  # [batch_size, 512].
        sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 512].
        return pooled_output, sequence_output

with tensorflow.keras.utils.custom_object_scope({'BertEncoder': BertEncoder}):
    # Load the model architecture and weights
    model = tensorflow.keras.models.load_model('/content/Model/model-bert-use-this.hdf5')

# Load model weights and optimizer state from pickle file
with open('/content/Model/model_weights_and_optimizer.pickle', 'rb') as f:
    weights, optimizer_config = pickle.load(f)

# Set model weights
model.set_weights(weights)

# Reconstruct optimizer from config
optimizer = tensorflow.keras.optimizers.Adam.from_config(optimizer_config)

print("Model Loaded")

Model Loaded


In [ ]:
y_main=[]
for i in questions:
  temp1=[]
  y_pred=model.predict(data_df[i].fillna("NA"))
  for j in y_pred:
    temp1.append(j[0])
  y_pred
  y_main.append(temp1)

8/8 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 23s 3s/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 23s 3s/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step


In [ ]:
y_main = [list(i) for i in zip(*y_main)]#transposing the matrix
ai_df=pd.DataFrame(y_main)

Natural Language Processing Below

In [ ]:
import pandas as pd
from joblib import load
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords'), nltk.download('wordnet')

def preprocess(text):
    if isinstance(text, str):
        return ' '.join([WordNetLemmatizer().lemmatize(word.lower()) for word in re.sub(r'\W+', ' ', text.lower()).split() if word.lower() not in stopwords.words('english')])
    else:
        return ''
def process_data(file_path):
    data_df = pd.read_excel(file_path).fillna('')
    data_df['Level1'] = data_df.iloc[:, level1_idx].apply(lambda x: ''.join(x.astype(str)), axis=1, result_type='expand')
    data_df['Level2'] = data_df.iloc[:, level2_idx].apply(lambda x: ''.join(x.astype(str)), axis=1, result_type='expand')
    all_cols_to_del = cols_to_del + level1_idx + level2_idx
    data_df.drop(data_df.columns[all_cols_to_del], axis=1, inplace=True)
    data_df['Level3'] = data_df.apply(lambda x: ''.join(x.astype(str)), axis=1, result_type='expand')
    data_df.drop(data_df.columns[[i for i, col in enumerate(data_df.columns) if col not in ['Level1', 'Level2','Level3']]], axis=1, inplace=True)
    data_df = data_df[['Level1', 'Level2', 'Level3']].applymap(preprocess).fillna('')
    return data_df

loaded_vectorizer, loaded_model = load('/content/Model/vectorizer.joblib'), load('/content/Model/random_forest_model.joblib')
data_df = process_data('/content/data.xlsx')
data_df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Level1,Level2,Level3
0,entrepreneurship process identifying problem o...,entrepreneurship process identifying problem o...,ambitious dreamer lazy streak striving greatne...
1,entrepreneurship mean courage turn idea realit...,entrepreneurship mean courage turn idea realit...,tweet would describe passionate learner eager ...
2,process identifying creating pursuing opportun...,process identifying creating pursuing opportun...,enthusiastic lad zest knowledge sporty spirit ...
3,mean express worth team group setting usually ...,mean express worth team group setting usually ...,good art ever since young age music something ...
4,lot connection able sway anyone built extended...,lot connection able sway anyone,enthusiast keen growing connection esteemed cl...
...,...,...,...
234,entrepreneurship mindset set action involve id...,entrepreneurship mindset set action involve id...,fun loving extroverted person passionate music...
235,entrepreneurship mean ability make something s...,entrepreneurship mean ability make something s...,name vishaal g fun adventurous person like pla...
236,entrepreneur mean finding something passionate...,entrepreneur mean finding something passionate...,priyanka iam hard workable person much patienc...
237,entrepreneurship embodies spirit innovation ri...,entrepreneurship embodies spirit innovation ri...,driven enthusiastic creative individual zest l...


In [ ]:
# Assume data_df is already defined and includes the required data
text_columns = data_df.columns

# List to store each row's data
data = []

for index, row in data_df.iterrows():
    # Vectorize the given row
    answers_row = ' '.join(row[text_columns].astype(str))
    answers_row_vec = loaded_vectorizer.transform([answers_row])

    # Predict the probability of the confirmed class
    probability = loaded_model.predict_proba(answers_row_vec)[:, 1]

    # Append dictionary to list
    data.append({'probability': float(probability[0])})

# Create DataFrame from list of dictionaries
results_df = pd.DataFrame(data)

print(results_df.head())

nlp_df = results_df

   probability
0     0.780000
1     0.279524
2     0.712500
3     0.602149
4     0.248950


Outputting to the Excel File

In [ ]:
import pandas as pd
import numpy as np


# I am dividing it here to normalize the score

ai_df.loc[:, "AI Component"] = ai_df.sum(axis=1) / 5
ai_df = ai_df[["AI Component"]]

temp_df = full_dataframe
temp_df = pd.concat([temp_df, ai_df, nlp_df], axis=1)
temp_df.to_excel("/content/output.xlsx", index=True)

print("Output File Created!")
from google.colab import files
files.download('/content/output.xlsx')

Output File Created!
